In [11]:
import pandas as pd
from datetime import datetime
import numpy as np

pd.set_option('display.max_columns', 100)
pd.set_option('display.max_rows', 100)

In [12]:
matchesBB1 = pd.read_csv('data/BBOrdersMatched240319.csv')
matchesETH1 = pd.read_csv('data/ETHOrdersMatched240314.csv')
queries = pd.read_csv('data/Finished Files/AddressQueries240328.csv')

In [13]:
# relabel
matchesBB1 = matchesBB1.rename(columns={'IN_ADDR':'CASS_Address', 
                                        'IN_CITY':'CASS_City', 'IN_STATE':'CASS_State', 
                                        'IN_ZIP':'CASS_ZIP'})

In [14]:
# match orders with addresses
matchesBB = queries.merge(matchesBB1, on = ['CASS_Address', 'CASS_City', 'CASS_State', 'CASS_ZIP'], how='right')
matchesBB['MATCHED'] = matchesBB['First Requested (B)'].notna()
matchesETH = queries.merge(matchesETH1, on = ['CASS_Address', 'CASS_City', 'CASS_State', 'CASS_ZIP'], how='right')
matchesETH['MATCHED'] = matchesETH['First Requested (E)'].notna()

In [15]:
# formatting columns
cols1 = ['First Requested (E)', 'Last Requested (E)', 'First Requested (B)', 'Last Requested (B)', 'POST_DT', 'SO_CREATE_DT']
matchesBB[cols1] = matchesBB[cols1].apply(pd.to_datetime)
cols2 = ['First Requested (E)', 'Last Requested (E)', 'First Requested (B)', 'Last Requested (B)', 'DD_COMPL_DT', 'ORIG_DATE_REC']
matchesETH[cols2] = matchesETH[cols2].apply(pd.to_datetime)
cols3 = ['First Requested (E)', 'Last Requested (E)', 'First Requested (B)', 'Last Requested (B)']
queries[cols3] = queries[cols3].apply(pd.to_datetime)
matchesETH['DOC_NUM'] = matchesETH['DOC_NUM'].astype('str')

In [16]:
# filtering out FOTS and VZON products
matchesETH = matchesETH[(matchesETH['ETH_PROD'] != 'FOTS') & (matchesETH['ETH_PROD'] != 'VZON')]

In [17]:
# dropping document number duplicates and filtering for EVC addresses over UNI if applicable
matchesETH = matchesETH.sort_values(by=['CIRCUIT_TYPE'], axis=0, ascending=True
                                    ).drop_duplicates(['DOC_NUM', 'CASS_Address', 'CASS_City', 'CASS_State', 
                                                       'CASS_ZIP'], keep = 'first')

In [18]:
# drop duplicates by address and order number
matchesBB = matchesBB.drop_duplicates(['CASS_Address', 'CASS_City', 'CASS_State', 'CASS_ZIP', 'ORDNO'])

In [26]:
# filter out bb matches that only have commercial orders (2988 records -> 1304 records)
matchesBB = matchesBB[(matchesBB['SERV_TYPE'] == 'BUS')]

In [27]:
# BB columns for differences between days
matchesBB['Days Between Order Date and First Requested (B)'] = matchesBB['SO_CREATE_DT'] - matchesBB['First Requested (B)']
matchesBB['Days Between Build and Order Date (B)'] = matchesBB['POST_DT'] - matchesBB['SO_CREATE_DT']

In [28]:
# ETH columns for differences between days
matchesETH['Days Between Order Date and First Requested (E)'] = matchesETH['ORIG_DATE_REC'] - matchesETH['First Requested (E)']
matchesETH['Days Between Build and Order Date (E)'] = matchesETH['DD_COMPL_DT'] - matchesETH['ORIG_DATE_REC']

In [29]:
# variable for grouping by month
matchesETH['GROUP_DATE'] = matchesETH['ORIG_DATE_REC'].dt.to_period('M')
matchesBB['GROUP_DATE'] = matchesBB['SO_CREATE_DT'].dt.to_period('M')

In [30]:
# new ethernet df with only matches
matchedETH = matchesETH[matchesETH['MATCHED'] == True]

In [31]:
# new broadband df with only matches
matchedBB = matchesBB[matchesBB['MATCHED'] == True]

In [33]:
matchedBB['Days Between Build and Order Date (B)'].value_counts(
).reset_index(
).sort_values(by='Days Between Build and Order Date (B)'
              ).head()

,Days Between Build and Order Date (B),count
43,2 days,1
34,3 days,1
20,4 days,2
14,5 days,3
27,6 days,1


In [34]:
matchedETH['Days Between Build and Order Date (E)'].value_counts(
).reset_index(
).sort_values(by='Days Between Build and Order Date (E)'
              ).head()

,Days Between Build and Order Date (E),count
22,0 days,4
1,1 days,21
2,2 days,16
4,3 days,14
5,4 days,14


In [137]:
# Average time from first query to BB Order (for orders matching CB/MS and that are positive)
tbbqo = matchedBB[matchedBB['Days Between Order Date and First Requested (B)'] >= 
        pd.Timedelta("0 days")]['Days Between Order Date and First Requested (B)'].mean()

In [138]:
# Average time from first query to ETH Order (for orders matching CB/MS and that are positive)
tethqo = matchedETH[matchedETH['Days Between Order Date and First Requested (E)'] >= 
        pd.Timedelta("0 days")]['Days Between Order Date and First Requested (E)'].mean()

In [139]:
# Average time from Order to Build BB (for orders matching CB/MS)
tbbob = matchedBB['Days Between Build and Order Date (B)'].mean()

In [48]:
matchedBB[matchedBB['Days Between Order Date and First Requested (B)'] >= 
        pd.Timedelta("0 days")]['Days Between Order Date and First Requested (B)'].value_counts()

Days Between Order Date and First Requested (B)
0 days      13
1 days       8
22 days      5
18 days      5
28 days      5
40 days      4
7 days       4
41 days      4
34 days      4
92 days      4
14 days      4
8 days       3
20 days      3
9 days       3
36 days      3
133 days     3
21 days      3
48 days      3
43 days      3
2 days       3
78 days      2
55 days      2
104 days     2
33 days      2
35 days      2
73 days      2
5 days       2
11 days      2
4 days       2
110 days     2
26 days      2
24 days      2
64 days      2
16 days      2
15 days      2
6 days       2
60 days      1
95 days      1
68 days      1
29 days      1
51 days      1
38 days      1
44 days      1
88 days      1
10 days      1
32 days      1
17 days      1
67 days      1
37 days      1
63 days      1
62 days      1
136 days     1
58 days      1
30 days      1
61 days      1
49 days      1
85 days      1
89 days      1
57 days      1
70 days      1
98 days      1
31 days      1
27 days      1
3 days 

In [140]:
# Average time from Order to Build ETH (for orders matching CB/MS)
tethob = matchedETH['Days Between Build and Order Date (E)'].mean()

In [141]:
# Average time from first query to Order BB(for orders after Oct. 1)
fbbqo = matchesBB[matchesBB['Days Between Order Date and First Requested (B)'] >= 
        pd.Timedelta("0 days")]['Days Between Order Date and First Requested (B)'].mean()

In [142]:
# Average time from first query to Order ETH (for orders after Oct. 1)
fethqo = matchesETH[matchesETH['Days Between Order Date and First Requested (E)'] >= 
        pd.Timedelta("0 days")]['Days Between Order Date and First Requested (E)'].mean()

In [143]:
foundOrderAverages = pd.DataFrame(data = {'Quote to Order Average (in days)' : [tbbqo.days, tethqo.days], 
                     'Order to Build Average (in days)' : [tbbob.days, tethob.days]}, 
                     index=['Broadband', 'Ethernet'])

In [144]:
countBB = queries.drop_duplicates(['CASS_Address','CASS_City','CASS_State',
                                   'CASS_ZIP','Days Between First and Last Request (B)']
                                   )['Days Between First and Last Request (B)'].notnull().sum()
countETH = queries.drop_duplicates(['CASS_Address','CASS_City','CASS_State',
                                   'CASS_ZIP','Days Between First and Last Request (E)']
                                   )['Days Between First and Last Request (E)'].notnull().sum()

In [145]:
# count of serviceable footprints
ETHfootprintCount = queries[queries['Footprint (E)'] == True].drop_duplicates(['CASS_Address','CASS_City','CASS_State',
                                   'CASS_ZIP','Days Between First and Last Request (E)']
                                   )['Days Between First and Last Request (E)'].notnull().sum()

In [146]:
# conversion rates from MS/CB to orders
conversion = pd.DataFrame(data = {
    'Query to Order Rate (%)' : [round((len(matchedBB)/countBB)*100, 2), 
                                 round((len(matchedETH)/countETH)*100, 2)],
    'Total Orders coming from CB/MS (%)': [round((len(matchedBB)/len(matchesBB))*100, 2), 
                                                  round((len(matchedETH)/len(matchesETH))*100, 2)], 
    'Orders Placed from CB/MS' : [len(matchedBB), len(matchedETH)],
    'Total Commercial Orders Placed' : [len(matchesBB), len(matchesETH)],
    'Unique Query Addresses' : [countBB, countETH],
    'Serviceable Query Count' : ['', ETHfootprintCount],
    'Serviceable Query to Order Rate (%)': ['', round((len(matchedETH)/ETHfootprintCount)*100, 2)],
    },
    index=['Broadband', 'Ethernet'])

In [147]:
bbmerge1 = matchedBB.groupby('GROUP_DATE'
                                     ).size(
                                     ).reset_index(
                                     ).rename(columns={'GROUP_DATE':'Month First Requested', 0:'Count of Matched Orders'}
                                              )
bbmerge2 = matchesBB.groupby('GROUP_DATE'
                                     ).size(
                                     ).reset_index(
                                     ).rename(columns={'GROUP_DATE':'Month First Requested', 0:'Count of Orders'}
                                              )

BBmatchesByMonth = bbmerge1.merge(bbmerge2, on = 'Month First Requested')
BBmatchesByMonth['% of Orders Attributed to CB/MS'] = round((BBmatchesByMonth['Count of Matched Orders'] / BBmatchesByMonth['Count of Orders'])*100, 2)

In [148]:
ethmerge1 = matchedETH.groupby('GROUP_DATE'
                                     ).size(
                                     ).reset_index(
                                     ).rename(columns={'GROUP_DATE':'Month First Requested', 0:'Count of Matched Orders'}
                                              )
ethmerge2 = matchesETH.groupby('GROUP_DATE'
                                     ).size(
                                     ).reset_index(
                                     ).rename(columns={'GROUP_DATE':'Month First Requested', 0:'Count of Orders'}
                                              )

ETHmatchesByMonth = ethmerge1.merge(ethmerge2, on = 'Month First Requested')
ETHmatchesByMonth['% of Orders Attributed to CB/MS'] = round((ETHmatchesByMonth['Count of Matched Orders'] / ETHmatchesByMonth['Count of Orders'])*100, 2)

In [149]:
# monthly query to order rates for broadband
BBqueries = queries.merge(matchedBB, on = ['CASS_Address', 'CASS_City', 'CASS_State', 'CASS_ZIP'], how='left')
BBqueries['GROUP_DATE'] = BBqueries['First Requested (B)_x'].dt.to_period('M')
BBqueries = BBqueries.dropna(subset = 'GROUP_DATE')
BBqueries['IS_QUERY'] = True
BBrates = BBqueries.groupby('GROUP_DATE'
                  )[['MATCHED', 'IS_QUERY']].sum(
                  ).reset_index(  
                  ).rename(columns={'GROUP_DATE': 'Month First Requested', 'MATCHED': 'Queries That Became an Order', 
                                    'IS_QUERY':'Total Queries'}
                                    )
BBrates['Queries That Became an Order'] = pd.to_numeric(BBrates['Queries That Became an Order'], errors='coerce')
BBrates['Total Queries'] = pd.to_numeric(BBrates['Total Queries'], errors='coerce')
BBrates['Query to Order Conversion Rate (%)'] = round((BBrates['Queries That Became an Order']/BBrates['Total Queries']) * 100, 2)

In [150]:
ETHqueries = queries.merge(matchedETH, on = ['CASS_Address', 'CASS_City', 'CASS_State', 'CASS_ZIP'], how='left')
ETHqueries['GROUP_DATE'] = ETHqueries['First Requested (E)_x'].dt.to_period('M')
ETHqueries['IS_QUERY'] = True
ETHqueries.groupby('GROUP_DATE'
                  )[['MATCHED', 'IS_QUERY', 'Footprint (E)_x']].sum(
                  ).reset_index(  
                  ).rename(columns={'GROUP_DATE': 'Month First Requested', 'MATCHED': 'Queries That Became an Order', 
                                    'IS_QUERY':'Total Queries', 'Footprint (E)_x':'Total Serviceable Queries'}
                                    )

,Month First Requested,Queries That Became an Order,Total Queries,Total Serviceable Queries
0,2023-10,168,11585,1766
1,2023-11,76,5548,738
2,2023-12,156,5602,1739
3,2024-01,198,8624,2910
4,2024-02,105,9003,2707
5,2024-03,2,1085,120


In [151]:
# monthly query to order rates for ethernet
ETHqueries = queries.merge(matchedETH, on = ['CASS_Address', 'CASS_City', 'CASS_State', 'CASS_ZIP'], how='left')
ETHqueries['GROUP_DATE'] = ETHqueries['First Requested (E)_x'].dt.to_period('M')
ETHqueries = ETHqueries.dropna(subset = 'GROUP_DATE')
ETHqueries['IS_QUERY'] = True
ETHrates = ETHqueries.groupby('GROUP_DATE'
                  )[['MATCHED', 'IS_QUERY', 'Footprint (E)_x']].sum(
                  ).reset_index(  
                  ).rename(columns={'GROUP_DATE': 'Month First Requested', 'MATCHED': 'Queries That Became an Order', 
                                    'IS_QUERY':'Total Queries', 'Footprint (E)_x':'Total Serviceable Queries'}
                                    )
ETHrates['Queries That Became an Order'] = pd.to_numeric(ETHrates['Queries That Became an Order'], errors='coerce')
ETHrates['Total Queries'] = pd.to_numeric(ETHrates['Total Queries'], errors='coerce')
ETHrates['Total Serviceable Queries'] = pd.to_numeric(ETHrates['Total Serviceable Queries'], errors='coerce')
ETHrates['Query to Order Conversion Rate (%)'] = round((ETHrates['Queries That Became an Order']/ETHrates['Total Queries']) * 100, 2)
ETHrates['Serviceable Query to Order Conversion Rate (%)'] = round((ETHrates['Queries That Became an Order']/ETHrates['Total Serviceable Queries']) * 100, 2)

In [152]:
ETHdaysToBuildByBuildType = matchesETH.groupby('BUILD_GRP')['Days Between Build and Order Date (E)'].mean(
                                     ).reset_index(
                                     ).rename(columns={'BUILD_GRP':'Build Type', 
                                                       'Days Between Build and Order Date (E)': 
                                                       'Average Days Between Build and Order Date (in days)'}
                                              )
ETHdaysToBuildByBuildType['Average Days Between Build and Order Date (in days)'] = ETHdaysToBuildByBuildType['Average Days Between Build and Order Date (in days)'].dt.days

In [153]:
# averages for orders that were found in query addresses
foundOrderAverages

,Quote to Order Average (in days),Order to Build Average (in days)
Broadband,35,20
Ethernet,55,24


In [154]:
# rates are for orders that were found in query addresses
conversion

,Query to Order Rate (%),Total Orders coming from CB/MS (%),Orders Placed from CB/MS,Total Commercial Orders Placed,Unique Query Addresses,Serviceable Query Count,Serviceable Query to Order Rate (%)
Broadband,0.38,15.57,203,1304,53865,,
Ethernet,1.70,18.77,703,3746,41363,9913,7.09


In [155]:
# matched orders
BBmatchesByMonth

,Month First Requested,Count of Matched Orders,Count of Orders,% of Orders Attributed to CB/MS
0,2023-10,16,207,7.73
1,2023-11,20,226,8.85
2,2023-12,26,212,12.26
3,2024-01,52,226,23.01
4,2024-02,63,287,21.95
5,2024-03,26,146,17.81


In [156]:
# matched orders
ETHmatchesByMonth

,Month First Requested,Count of Matched Orders,Count of Orders,% of Orders Attributed to CB/MS
0,2023-10,75,656,11.43
1,2023-11,72,530,13.58
2,2023-12,90,652,13.80
3,2024-01,163,717,22.73
4,2024-02,209,831,25.15
5,2024-03,94,360,26.11


In [157]:
# total orders
ETHdaysToBuildByBuildType

,Build Type,Average Days Between Build and Order Date (in days)
0,Build,57
1,PortAdd/NoBuild,17
2,TBD,5


In [158]:
BBrates

,Month First Requested,Queries That Became an Order,Total Queries,Query to Order Conversion Rate (%)
0,2023-10,13,7974,0.16
1,2023-11,23,10371,0.22
2,2023-12,40,6509,0.61
3,2024-01,93,14268,0.65
4,2024-02,32,13013,0.25
5,2024-03,2,1757,0.11


In [159]:
ETHrates

,Month First Requested,Queries That Became an Order,Total Queries,Total Serviceable Queries,Query to Order Conversion Rate (%),Serviceable Query to Order Conversion Rate (%)
0,2023-10,168,11585,1766,1.45,9.51
1,2023-11,76,5548,738,1.37,10.30
2,2023-12,156,5602,1739,2.78,8.97
3,2024-01,198,8624,2910,2.30,6.80
4,2024-02,105,9003,2707,1.17,3.88
5,2024-03,2,1085,120,0.18,1.67


In [160]:
temp = pd.read_csv('data/Finished Files/LatLong240322.csv')[['ID', 'LAT', 'LON']]
temp['ID'] = temp['ID'].astype('str')
temp1 = matchedETH.loc[matchedETH['Footprint (E)'].isnull(), 'ID':'CASS_ZIP']#.to_csv('OutOfFootprintEthernetOrders.csv', index=False)
temp1['ID'] = temp1['ID'].astype('str').str.split('.').str[0]
temp1.merge(temp, on = 'ID', how = 'left').to_csv('OutOfFootprintEthernetOrders.csv', index=False)

C:\Users\sss7371\AppData\Local\Temp\ipykernel_17712\3380764491.py:1: DtypeWarning: Columns (14,15,16) have mixed types. Specify dtype option on import or set low_memory=False.
  temp = pd.read_csv('data/Finished Files/LatLong240322.csv')[['ID', 'LAT', 'LON']]


In [161]:
temp1['ID'].astype('str').str.split('.').str[0]

3700    40546
5780    30606
2919    30530
3195    34433
6056    24661
4406    25809
4424    21131
4269     5843
4226    30530
4828    25460
4665     9153
5473    20807
3796    21617
3780    34715
5509    24661
5308    21824
3887    22766
6825     3433
781      1831
1264    22835
6612    31317
256     33864
7141    21287
7235    28776
2       12192
6391     5843
2074    37066
Name: ID, dtype: object

In [162]:
temp[['ID', 'LAT', 'LON']]

,ID,LAT,LON
0,179,NaN,NaN
1,77895,NaN,NaN
2,3020,NaN,NaN
3,62525,NaN,NaN
4,73498,NaN,NaN
...,...,...,...
78519,41669,NaN,NaN
78520,41696,NaN,NaN
78521,40332,NaN,NaN
78522,40337,NaN,NaN
